##

In [2]:
import sagemaker
import boto3
import os

In [3]:
import sagemaker
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

from sagemaker.pytorch import PyTorch

In [17]:
from sagemaker.debugger import Rule, ProfilerRule, rule_configs
from sagemaker.debugger import DebuggerHookConfig, ProfilerConfig, FrameworkProfile
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts
import IPython

[2023-03-28 19:27:18.292 1-8-1-cpu-py36-ml-t3-medium-b1e105fab0ca332569ae17f8727f:49 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None


In [4]:
bucket= 'dog-images-mle'
print("Default Bucket: {}".format(bucket))

region = 'us-east-1'## TODO: fill in
print("AWS Region: {}".format(region))

role = sagemaker.get_execution_role()## TODO: fill in
print("RoleArn: {}".format(role))

Default Bucket: dog-images-mle
AWS Region: us-east-1
RoleArn: arn:aws:iam::085616114435:role/service-role/AmazonSageMaker-ExecutionRole-20221221T205295


In [18]:
rules = [
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
]

from sagemaker.debugger import DebuggerHookConfig, ProfilerConfig, FrameworkProfile

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10)
)
debugger_config = DebuggerHookConfig(
    hook_parameters={"train.save_interval": "100", "eval.save_interval": "10"}
)

In [19]:
estimator = PyTorch(
    entry_point='train_model.py',
    role=role,
    instance_count=1,
    instance_type="ml.m5.4xlarge",
    #hyperparameters=hyperparameters,
    framework_version="1.8",
    profiler_config=profiler_config,
    debugger_hook_config=debugger_config,
    rules=rules,
    py_version="py36",
    source_dir="code"
)

In [20]:
estimator.fit({'train':'s3://udacity-mle-capstone/train/',
               'test':'s3://udacity-mle-capstone/test/',
               'meta':'s3://udacity-mle-capstone/meta/'
              }, 
              wait=True)

2023-03-28 19:29:22 Starting - Starting the training job...
2023-03-28 19:29:38 Starting - Preparing the instances for trainingLossNotDecreasing: InProgress
VanishingGradient: InProgress
Overfit: InProgress
Overtraining: InProgress
PoorWeightInitialization: InProgress
LowGPUUtilization: InProgress
ProfilerReport: InProgress
......
2023-03-28 19:30:47 Downloading - Downloading input data...
2023-03-28 19:31:21 Training - Downloading the training image...
2023-03-28 19:31:48 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-03-28 19:31:44,437 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-03-28 19:31:44,440 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-03-28 19:31:44,448 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-03-

In [23]:
training_job_name = estimator.latest_training_job.name
print(f"Training jobname: {training_job_name}")
print(f"Region: {region}")

trial = create_trial(estimator.latest_job_debugger_artifacts_path())

print(trial.tensor_names())
#print(len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.TRAIN)))
#print(len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.EVAL)))

tj = TrainingJob(training_job_name, 'us-east-2')
tj.wait_for_sys_profiling_data_to_be_available()

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU"],
    select_events=["total"],
)

rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")

! aws s3 ls {rule_output_path} --recursive
! aws s3 cp {rule_output_path} ./ --recursive

# get the autogenerated folder name of profiler report
profiler_report_name = [
    rule["RuleConfigurationName"]
    for rule in estimator.latest_training_job.rule_job_summary()
    if "Profiler" in rule["RuleConfigurationName"]
][0]

IPython.display.HTML(filename=profiler_report_name + "/profiler-output/profiler-report.html")

Training jobname: pytorch-training-2023-03-28-19-29-21-822
Region: us-east-1
[2023-03-28 20:35:48.779 1-8-1-cpu-py36-ml-t3-medium-b1e105fab0ca332569ae17f8727f:49 INFO s3_trial.py:42] Loading trial debug-output at path s3://sagemaker-us-east-2-085616114435/pytorch-training-2023-03-28-19-29-21-822/debug-output
[2023-03-28 20:35:49.052 1-8-1-cpu-py36-ml-t3-medium-b1e105fab0ca332569ae17f8727f:49 INFO trial.py:198] Training has ended, will refresh one final time in 1 sec.
[2023-03-28 20:35:50.071 1-8-1-cpu-py36-ml-t3-medium-b1e105fab0ca332569ae17f8727f:49 INFO trial.py:210] Loaded all steps
['BCELoss_output_0', 'gradient/SiameseNetwork_fc.0.bias', 'gradient/SiameseNetwork_fc.0.weight', 'gradient/SiameseNetwork_fc.2.bias', 'gradient/SiameseNetwork_fc.2.weight', 'gradient/SiameseNetwork_resnet.0.weight', 'gradient/SiameseNetwork_resnet.1.bias', 'gradient/SiameseNetwork_resnet.1.weight', 'gradient/SiameseNetwork_resnet.4.0.bn1.bias', 'gradient/SiameseNetwork_resnet.4.0.bn1.weight', 'gradient/S

You will find the profiler report in s3://sagemaker-us-east-2-085616114435/pytorch-training-2023-03-28-19-29-21-822/rule-output
2023-03-28 20:27:44     386481 pytorch-training-2023-03-28-19-29-21-822/rule-output/ProfilerReport/profiler-output/profiler-report.html
2023-03-28 20:27:43     236531 pytorch-training-2023-03-28-19-29-21-822/rule-output/ProfilerReport/profiler-output/profiler-report.ipynb
2023-03-28 20:27:39        192 pytorch-training-2023-03-28-19-29-21-822/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json
2023-03-28 20:27:39        200 pytorch-training-2023-03-28-19-29-21-822/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json
2023-03-28 20:27:39       1826 pytorch-training-2023-03-28-19-29-21-822/rule-output/ProfilerReport/profiler-output/profiler-reports/Dataloader.json
2023-03-28 20:27:39        127 pytorch-training-2023-03-28-19-29-21-822/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.jso

## load model

In [24]:
estimator.jobs[-1].describe()

{'TrainingJobName': 'pytorch-training-2023-03-28-19-29-21-822',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-2:085616114435:training-job/pytorch-training-2023-03-28-19-29-21-822',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-2-085616114435/pytorch-training-2023-03-28-19-29-21-822/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'sagemaker_container_log_level': '20',
  'sagemaker_job_name': '"pytorch-training-2023-03-28-19-29-21-822"',
  'sagemaker_program': '"train_model.py"',
  'sagemaker_region': '"us-east-2"',
  'sagemaker_submit_directory': '"s3://sagemaker-us-east-2-085616114435/pytorch-training-2023-03-28-19-29-21-822/source/sourcedir.tar.gz"'},
 'AlgorithmSpecification': {'TrainingImage': '763104351884.dkr.ecr.us-east-2.amazonaws.com/pytorch-training:1.8-cpu-py36',
  'TrainingInputMode': 'File',
  'EnableSageMakerMetricsTimeSeries': True},
 'RoleArn': 'arn:aws:iam::085616114435:role/service-role/A

In [25]:
s3_model_artifacts_uri = estimator.jobs[0].describe()['ModelArtifacts']['S3ModelArtifacts']

In [26]:
s3_model_artifacts_uri

's3://sagemaker-us-east-2-085616114435/pytorch-training-2023-03-28-19-29-21-822/output/model.tar.gz'

In [29]:
!aws s3 cp s3://sagemaker-us-east-2-085616114435/pytorch-training-2023-03-28-19-29-21-822/output/model.tar.gz /artifacts

download: s3://sagemaker-us-east-2-085616114435/pytorch-training-2023-03-28-19-29-21-822/output/model.tar.gz to ../../../artifacts


In [33]:
import sys

In [40]:
sys.path.insert(1,'/home/sagemaker-user/udacity-mle-capstone/project/artifacts/')

In [46]:
import torch.nn as nn
import torchvision
import torch

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # get resnet model
        self.resnet = torchvision.models.resnet18()
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.fc_in_features = self.resnet.fc.in_features
        
        # remove the last layer of resnet18 (linear layer which is before avgpool layer)
        self.resnet = torch.nn.Sequential(*(list(self.resnet.children())[:-1]))

        # add linear layers to compare between the features of the two images
        self.fc = nn.Sequential(
            nn.Linear(self.fc_in_features * 2, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 1),
        )

        self.sigmoid = nn.Sigmoid()

        # initialize the weights
        self.resnet.apply(self.init_weights)
        self.fc.apply(self.init_weights)
        
    def init_weights(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)

    def forward_once(self, x):
        output = self.resnet(x)
        output = output.view(output.size()[0], -1)
        return output

    def forward(self, input1, input2):
        input1 = input1.view(-1, 1, 150, 220).float().div(255)
        input2 = input2.view(-1, 1, 150, 220).float().div(255)
        # get two images' features
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)

        # concatenate both images' features
        output = torch.cat((output1, output2), 1)

        # pass the concatenation to the linear layers
        output = self.fc(output)

        # pass the out of the linear layers to sigmoid layer
        output = self.sigmoid(output)
        
        return output

In [47]:
model = SiameseNetwork()

In [50]:
model.load_state_dict(torch.load('/root/udacity-mle-capstone/project/artifacts/siamese_network.pt'))
#model.eval()

<All keys matched successfully>

In [51]:
model.eval()

SiameseNetwork(
  (resnet): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats

In [53]:
!pip install torchviz

  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 46 kB 665 kB/s             
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4149 sha256=5be6a11c40671f0caa39b4cd1464ddba1b4a38c19d31c9beab3b86301dc05185
  Stored in directory: /root/.cache/pip/wheels/21/c4/af/3efa088be2f95b4953ed4bc8e39a4a95a82d19a7134a4c7a1b
Successfully built torchviz


In [57]:
!pip install scikit-image

     |████████████████████████████████| 12.4 MB 24.1 MB/s            
     |████████████████████████████████| 4.4 MB 69.1 MB/s            
     |████████████████████████████████| 148 kB 84.6 MB/s            


In [60]:
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from skimage import filters, transform
from skimage.io import imread
from skimage import img_as_ubyte
from typing import Tuple

from torch.utils.data import Dataset
from torchvision import datasets

import pandas as pd
import numpy as np


def preprocess_signature(img: np.ndarray,
                         canvas_size: Tuple[int, int],
                         img_size: Tuple[int, int] =(170, 242),
                         input_size: Tuple[int, int] =(150, 220)) -> np.ndarray:
    img = img.astype(np.uint8)
    centered = normalize_image(img, canvas_size)
    inverted = 255 - centered
    resized = resize_image(inverted, img_size)

    if input_size is not None and input_size != img_size:
        cropped = crop_center(resized, input_size)
    else:
        cropped = resized

    return cropped


def normalize_image(img: np.ndarray,
                    canvas_size: Tuple[int, int] = (840, 1360)) -> np.ndarray:

    # 1) Crop the image before getting the center of mass

    # Apply a gaussian filter on the image to remove small components
    # Note: this is only used to define the limits to crop the image
    blur_radius = 2
    blurred_image = filters.gaussian(img, blur_radius, preserve_range=True)

    # Binarize the image using OTSU's algorithm. This is used to find the center
    # of mass of the image, and find the threshold to remove background noise
    threshold = filters.threshold_otsu(img)

    # Find the center of mass
    binarized_image = blurred_image > threshold
    r, c = np.where(binarized_image == 0)
    r_center = int(r.mean() - r.min())
    c_center = int(c.mean() - c.min())

    # Crop the image with a tight box
    cropped = img[r.min(): r.max(), c.min(): c.max()]

    # 2) Center the image
    img_rows, img_cols = cropped.shape
    max_rows, max_cols = canvas_size

    r_start = max_rows // 2 - r_center
    c_start = max_cols // 2 - c_center

    # Make sure the new image does not go off bounds
    # Emit a warning if the image needs to be cropped, since we don't want this
    # for most cases (may be ok for feature learning, so we don't raise an error)
    if img_rows > max_rows:
        # Case 1: image larger than required (height):  Crop.
        print('Warning: cropping image. The signature should be smaller than the canvas size')
        r_start = 0
        difference = img_rows - max_rows
        crop_start = difference // 2
        cropped = cropped[crop_start:crop_start + max_rows, :]
        img_rows = max_rows
    else:
        extra_r = (r_start + img_rows) - max_rows
        # Case 2: centering exactly would require a larger image. relax the centering of the image
        if extra_r > 0:
            r_start -= extra_r
        if r_start < 0:
            r_start = 0

    if img_cols > max_cols:
        # Case 3: image larger than required (width). Crop.
        print('Warning: cropping image. The signature should be smaller than the canvas size')
        c_start = 0
        difference = img_cols - max_cols
        crop_start = difference // 2
        cropped = cropped[:, crop_start:crop_start + max_cols]
        img_cols = max_cols
    else:
        # Case 4: centering exactly would require a larger image. relax the centering of the image
        extra_c = (c_start + img_cols) - max_cols
        if extra_c > 0:
            c_start -= extra_c
        if c_start < 0:
            c_start = 0

    normalized_image = np.ones((max_rows, max_cols), dtype=np.uint8) * 255
    # Add the image to the blank canvas
    normalized_image[r_start:r_start + img_rows, c_start:c_start + img_cols] = cropped

    # Remove noise - anything higher than the threshold. Note that the image is still grayscale
    normalized_image[normalized_image > threshold] = 255

    return normalized_image


def remove_background(img: np.ndarray) -> np.ndarray:

        img = img.astype(np.uint8)
        # Binarize the image using OTSU's algorithm. This is used to find the center
        # of mass of the image, and find the threshold to remove background noise
        threshold = filters.threshold_otsu(img)

        # Remove noise - anything higher than the threshold. Note that the image is still grayscale
        img[img > threshold] = 255

        return img


def resize_image(img: np.ndarray,
                 size: Tuple[int, int]) -> np.ndarray:
    height, width = size

    # Check which dimension needs to be cropped
    # (assuming the new height-width ratio may not match the original size)
    width_ratio = float(img.shape[1]) / width
    height_ratio = float(img.shape[0]) / height
    if width_ratio > height_ratio:
        resize_height = height
        resize_width = int(round(img.shape[1] / height_ratio))
    else:
        resize_width = width
        resize_height = int(round(img.shape[0] / width_ratio))

    # Resize the image (will still be larger than new_size in one dimension)
    img = transform.resize(img, (resize_height, resize_width),
                           mode='constant', anti_aliasing=True, preserve_range=True)

    img = img.astype(np.uint8)

    # Crop to exactly the desired new_size, using the middle of the image:
    if width_ratio > height_ratio:
        start = int(round((resize_width-width)/2.0))
        return img[:, start:start + width]
    else:
        start = int(round((resize_height-height)/2.0))
        return img[start:start + height, :]


def crop_center(img: np.ndarray,
                size: Tuple[int, int]) -> np.ndarray:
    img_shape = img.shape
    start_y = (img_shape[0] - size[0]) // 2
    start_x = (img_shape[1] - size[1]) // 2
    cropped = img[start_y: start_y + size[0], start_x:start_x + size[1]]
    return cropped


def crop_center_multiple(imgs: np.ndarray,
                         size: Tuple[int, int]) -> np.ndarray:
    img_shape = imgs.shape[2:]
    start_y = (img_shape[0] - size[0]) // 2
    start_x = (img_shape[1] - size[1]) // 2
    cropped = imgs[:, :, start_y: start_y + size[0], start_x:start_x + size[1]]
    return cropped

def load_signature(path):
    return img_as_ubyte(imread(path, as_gray=True))


class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # get resnet model
        self.resnet = torchvision.models.resnet18()

        # over-write the first conv layer to be able to read MNIST images
        # as resnet18 reads (3,x,x) where 3 is RGB channels
        # whereas MNIST has (1,x,x) where 1 is a gray-scale channel
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.fc_in_features = self.resnet.fc.in_features
        
        # remove the last layer of resnet18 (linear layer which is before avgpool layer)
        self.resnet = torch.nn.Sequential(*(list(self.resnet.children())[:-1]))

        # add linear layers to compare between the features of the two images
        self.fc = nn.Sequential(
            nn.Linear(self.fc_in_features * 2, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 1),
        )

        self.sigmoid = nn.Sigmoid()

        # initialize the weights
        self.resnet.apply(self.init_weights)
        self.fc.apply(self.init_weights)
        
    def init_weights(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)

    def forward_once(self, x):
        output = self.resnet(x)
        output = output.view(output.size()[0], -1)
        return output

    def forward(self, input1, input2):
        input1 = input1.view(-1, 1, 150, 220).float().div(255)
        input2 = input2.view(-1, 1, 150, 220).float().div(255)
        # get two images' features
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)

        # concatenate both images' features
        output = torch.cat((output1, output2), 1)

        # pass the concatenation to the linear layers
        output = self.fc(output)

        # pass the out of the linear layers to sigmoid layer
        output = self.sigmoid(output)
        
        return output

class SignatureDataset(Dataset):
    
    def __init__(self, category, canvas_size, dim=(256, 256)):
        df = pd.read_csv(f'/root/udacity-mle-capstone/project/sign_data/{category}_data.csv', header=0, names=['img_path_real', 'img_path_forged', 'label'])
        df['img_path_real'] = df['img_path_real'].apply(lambda x : f'/root/udacity-mle-capstone/project/sign_data/{category}/{x}')
        df['img_path_forged'] = df['img_path_forged'].apply(lambda x : f'/root/udacity-mle-capstone/project/sign_data/{category}/{x}')
        self.df  = df
        self.real_file_names = df["img_path_real"].values
        self.forged_file_names = df["img_path_forged"].values
        self.labels = df["label"].values
        self.dim = dim
        self.canvas_size=canvas_size

    def __len__(self):
        return len(self.df)
        
    def __getitem__(self,index):
        # getting the image path
        real_file_path = self.real_file_names[index]
        forged_file_path = self.forged_file_names[index]
        
        img1 = load_signature(real_file_path)
        img2 = load_signature(forged_file_path)
        
        img1 = preprocess_signature(img1, self.canvas_size, self.dim)
        img2 = preprocess_signature(img2, self.canvas_size, self.dim)
        
        label = torch.tensor(self.labels[index], dtype=torch.long)
        
        return torch.tensor(img1), torch.tensor(img2), label.float()

In [61]:
from torchviz import make_dot

In [62]:
train_dataset = SignatureDataset('train', (952, 1360))
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=64)

In [63]:
batch = next(iter(train_loader))
yhat = model(batch[0],batch[1])

[2023-03-28 21:22:17.334 1-8-1-cpu-py36-ml-t3-medium-b1e105fab0ca332569ae17f8727f:49 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
